In [ ]:
import pandas as pd
import numpy as np

import xgboost
import scipy.stats as stats

import lightgbm as lgb


from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


from matplotlib import pyplot as plt

from datetime import datetime
import ephem

from utils import smape, plot_eval

SEED = 42


In [ ]:
df = pd.read_csv('new_train.csv')
df.head()

In [ ]:
def prepare_data(df):
    df = df.drop(['lanes', 'lanes:forward', 'lit'], axis=1)

    unique_maxspeed = df['maxspeed'].unique()
    max_speed = []
    for data in tqdm(df['maxspeed'].to_numpy(), desc='Convert to kph'):
        if data == '30 mph':
            max_speed.append(48.2803)
        elif data == '20 mph':
            max_speed.append(32.1869)

    df['maxspeed'] = max_speed
    
    is_weekend = []
    is_night = []
    is_rush_hour = []
    date = []
    hour = []

    uk_observer = ephem.Observer()
    uk_observer.lat = '51.5074'  # Latitude of London
    uk_observer.lon = '-0.1278'  # Longitude of London


    for data in tqdm(df['waktu_setempat'].to_numpy(), desc='time categorization'):
        datetime_obj = datetime.strptime(data, '%Y-%m-%d %H:%M:%S%z')
        day_of_week = datetime_obj.weekday()
        if day_of_week >= 5:
            is_weekend.append(1)
        else:
            is_weekend.append(0)
        date_component = datetime_obj.strftime('%Y-%m-%d')
        hour_component = datetime_obj.strftime('%H')
        date.append(date_component)
        hour.append(int(hour_component))
        # Set the observer's date and time to the input UTC time
        uk_observer.date = datetime_obj

        # Calculate sunrise and sunset times
        sunrise = uk_observer.previous_rising(ephem.Sun())
        sunset = uk_observer.next_setting(ephem.Sun())
        if sunrise < uk_observer.date < sunset:
            is_night.append(0)
        else:
            is_night.append(1)
        
        # Define the time ranges
        morning_rush_hour_start = datetime.strptime('10:00:00', '%H:%M:%S').time()
        morning_rush_hour_end = datetime.strptime('16:00:00', '%H:%M:%S').time()

        night_rush_hour_start = datetime.strptime('20:00:00', '%H:%M:%S').time()
        night_rush_hour_end = datetime.strptime('23:59:59', '%H:%M:%S').time()

        night_rush_hour_start_2 = datetime.strptime('00:00:00', '%H:%M:%S').time()
        night_rush_hour_end_2 = datetime.strptime('06:00:00', '%H:%M:%S').time()

        # Extract the time component from the input datetime object
        input_time = datetime_obj.time()

        # Check if the time falls within the desired ranges
        if morning_rush_hour_start <= input_time <= morning_rush_hour_end or (night_rush_hour_start <= input_time <= night_rush_hour_end) or (night_rush_hour_start_2 <= input_time <= night_rush_hour_end_2):
            is_rush_hour.append(1)
        else:
            is_rush_hour.append(0)

    df['is_weekend'] = is_weekend
    df['hour'] = hour
    df['date'] = date
    df['is_night'] = is_night
    df['is_rush_hour'] = is_rush_hour
    df = df.drop(['waktu_setempat'], axis=1)
    
    return df
    

In [ ]:
df = prepare_data(df)
df.head()

In [ ]:
df['date'].unique()

In [ ]:
def plot_qq(data1, data2):
    sorted_data1 = np.sort(data1)
    sorted_data2 = np.sort(data2)
    n = len(sorted_data1)
    quantiles1 = np.arange(1, n + 1) / n
    quantiles2 = np.arange(1, n + 1) / n
    plt.figure(figsize=(6, 6))
    plt.scatter(sorted_data1, sorted_data2)
    plt.xlabel("Data rerata_kecepatan tanggal 2020-02-01")
    plt.ylabel("Data rerata_kecepatan tanggal 2020-02-02")
    plt.title("QQ Plot antara Data rerata_kecepatan tanggal 2020-02-01 dan 2020-02-02")
    plt.plot([min(sorted_data1), max(sorted_data1)], [min(sorted_data1), max(sorted_data1)], color='red', linestyle='--')

    plt.tight_layout()
    plt.show()

In [ ]:
for date in df['date'].unique():
    print(df[df['date'] == date].shape)

In [ ]:
df[df['date'] == '2020-02-01']['rerata_kecepatan'].to_numpy().shape
df[df['date'] == '2020-02-06']['rerata_kecepatan'].to_numpy().shape

In [ ]:

plot_qq(df[df['date'] == '2020-02-01']['rerata_kecepatan'].to_numpy(), df[df['date'] == '2020-02-02']['rerata_kecepatan'].to_numpy()[:19827])

In [ ]:
X = df.drop(['id_jalan', 'id_titik_mulai', 'id_titik_akhir', 'date', 'rerata_kecepatan'], axis=1)
y = df['rerata_kecepatan'].to_numpy()
# max_speed = y.max() * 1.2
# y = y / max_speed

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, shuffle=True)

In [ ]:
model = xgboost.XGBRegressor(n_jobs=-1, random_state=42)
model.fit(x_train, y_train)


In [ ]:
pred = model.predict(x_test)
print(f'test_data: {smape(y_test, pred)}')
plot_eval(pred*max_speed, y_test*max_speed)

In [ ]:
pred = model.predict(x_train)
print(f'train_data: {smape(y_train, pred)}')
plot_eval(pred*max_speed, y_train*max_speed)

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    
    
    
    n_estimators = trial.suggest_int("n_estimators", 1000, 5000)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    gamma = trial.suggest_float("gamma", 0.1, 1.0, step=0.1)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 7, step=2)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 0., 1.0, step=0.1)
    reg_lambda = trial.suggest_float("reg_lambda", 0., 1.0, step=0.1)
    seed = trial.suggest_int("random_state", 20, 50, step=2)
    
    
    model = xgboost.XGBRegressor(n_estimators=n_estimators,
                                max_depth=max_depth,
                                learning_rate=learning_rate,
                                gamma=gamma,
                                min_child_weight=min_child_weight,
                                colsample_bytree=colsample_bytree,
                                subsample=subsample,
                                reg_alpha=reg_alpha,
                                reg_lambda=reg_lambda,
                                n_jobs=-1, metric=mean_squared_error,
                                eval_metric=mean_squared_error,
                                random_state=seed
                                )
    
    model.fit(x_train, y_train)
    
    y_hat = model.predict(x_test)
    
    return mean_squared_error(y_test, y_hat, squared=True)

study = optuna.create_study()
study.optimize(objective, n_trials=50)

[I 2023-08-10 13:54:53,556] Trial 28 finished with value: 12.492314534341407 and parameters: {'n_estimators': 1587, 'max_depth': 8, 'learning_rate': 0.0415545742214241, 'gamma': 0.6, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.8, 'reg_alpha': 0.0, 'reg_lambda': 0.9, 'random_state': 28}. Best is trial 28 with value: 12.492314534341407.
[I 2023-08-10 04:37:44,166] Trial 32 finished with value: 12.503876898303945 and parameters: {'n_estimators': 2830, 'max_depth': 8, 'learning_rate': 0.02149679605911237, 'gamma': 0.1, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.5, 'reg_lambda': 0.6000000000000001, 'random_state': 42}. Best is trial 32 with value: 12.503876898303945.
[I 2023-08-10 05:53:02,056] Trial 42 finished with value: 12.490182406993412 and parameters: {'n_estimators': 3888, 'max_depth': 7, 'learning_rate': 0.024530410097984914, 'gamma': 0.1, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.5, 'random_state': 42}. Best is trial 42 with value: 12.490182406993412.


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    
    
    subsample_for_bin = trial.suggest_int("subsample_for_bin", 100000, 300000)
    n_estimators = trial.suggest_int("n_estimators", 1000, 5000)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    num_leaves = trial.suggest_int("num_leaves", 10, 50)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 7, step=2)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 0., 1.0, step=0.1)
    reg_lambda = trial.suggest_float("reg_lambda", 0., 1.0, step=0.1)
    seed = trial.suggest_int("random_state", 20, 50, step=2)
    min_child_samples = trial.suggest_int("min_child_samples", 10, 50)
    
    model = lgb.LGBMRegressor(num_leaves=num_leaves, max_depth=max_depth, learning_rate=learning_rate, 
                              n_estimators=n_estimators, subsample_for_bin=subsample_for_bin, 
                              min_child_weight=min_child_weight, min_child_samples=min_child_samples, subsample=subsample, 
                              colsample_bytree=colsample_bytree, reg_alpha=reg_alpha, 
                              reg_lambda=reg_lambda, random_state=seed, n_jobs=-1,)
    
    model.fit(x_train, y_train)
    
    y_hat = model.predict(x_test)
    
    return mean_squared_error(y_test, y_hat, squared=True)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

[I 2023-08-09 16:50:04,570] Trial 78 finished with value: 12.534454690626298 and parameters: {'subsample_for_bin': 155546, 'n_estimators': 3768, 'max_depth': 7, 'learning_rate': 0.04067420520019853, 'num_leaves': 50, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 1.0, 'reg_alpha': 0.5, 'reg_lambda': 0.30000000000000004, 'random_state': 34, 'min_child_samples': 16}. Best is trial 78 with value: 12.534454690626298.
[I 2023-08-09 16:58:36,247] Trial 93 finished with value: 12.533047460664628 and parameters: {'subsample_for_bin': 131673, 'n_estimators': 4510, 'max_depth': 6, 'learning_rate': 0.054834260577325884, 'num_leaves': 47, 'min_child_weight': 1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.7000000000000001, 'reg_lambda': 0.30000000000000004, 'random_state': 34, 'min_child_samples': 16}. Best is trial 93 with value: 12.533047460664628.
[I 2023-08-10 10:37:54,202] Trial 153 finished with value: 12.532367782002835 and parameters: {'subsample_for_bin': 150438, 'n_estimators': 2301, 'max_depth': 10, 'learning_rate': 0.06770102677206172, 'num_leaves': 49, 'min_child_weight': 3, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.7000000000000001, 'reg_lambda': 0.8, 'random_state': 38, 'min_child_samples': 26}. Best is trial 153 with value: 12.532367782002835.



In [ ]:
from catboost import CatBoostRegressor
import optuna



def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_float("l2_leaf_reg", 0., 1.0, step=0.1)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = trial.suggest_int("iterations", 1000, 5000)
    param['loss_function'] = 'RMSE'

    param['random_state'] = trial.suggest_int("random_state", 20, 50, step=2)
    param['logging_level'] = 'Silent'
    
    model = CatBoostRegressor(**param)

    model.fit(x_train, y_train)
    
    y_hat = model.predict(x_test)
    
    return mean_squared_error(y_test, y_hat, squared=True)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
def xgb_1(no):
  param_1 = {'n_estimators': 2552, 'max_depth': 9, 'learning_rate': 0.010419707354527082, 'gamma': 0.30000000000000004, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.9, 'reg_alpha': 0.30000000000000004, 'reg_lambda': 0.5, 'random_state': 28}

  param_2 = {'n_estimators': 4627, 'max_depth': 10, 
             'learning_rate': 0.03031698197768738, 
             'gamma': 0.7000000000000001, 'min_child_weight': 7, 
             'subsample': 0.9, 'colsample_bytree': 1.0, 
             'reg_alpha': 0.30000000000000004, 
             'reg_lambda': 0.30000000000000004, 
             'random_state': 22, 'grow_policy' : 'depthwise', 
            'n_jobs' : -1}
  
  params = [param_1, param_2]
  xgb = xgboost.XGBRegressor(**params[no])
  return xgb

def lgbm_1(no):
  param_1 = {'subsample_for_bin': 131673, 'n_estimators': 4510, 'max_depth': 6, 'learning_rate': 0.054834260577325884, 'num_leaves': 47, 'min_child_weight': 1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.7000000000000001, 'reg_lambda': 0.30000000000000004, 'random_state': 34, 'min_child_samples': 16}

  param_2 = {'subsample_for_bin': 155546, 'n_estimators': 3768, 'max_depth': 7, 'learning_rate': 0.04067420520019853, 'num_leaves': 50, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 1.0, 'reg_alpha': 0.5, 'reg_lambda': 0.30000000000000004, 'random_state': 34, 'min_child_samples': 16}
  
  params = [param_1, param_2]
  lgbm = lgb.LGBMRegressor(**params[no])
  return lgbm

In [ ]:
estimators1 = [('xgb_1{}'.format(i), xgb_1(i)) for i in range(1)]
estimators2 = [('lgbm_1{}'.format(j), lgbm_1(j))  for j in range(2)]
estimators = estimators1+estimators2


In [ ]:
for est in estimators:
    name, reg = est
    print(f'Evaluating {name} model')
    reg.fit(x_train, y_train)
    pred = reg.predict(x_test)
    print(f'test_data: {smape(y_test, pred)}')
    pred = reg.predict(x_train)
    print(f'train_data: {smape(y_train, pred)}')
    print('\n')

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

model = StackingRegressor(estimators=estimators, cv=3, final_estimator=LinearRegression(n_jobs=-1), n_jobs = -1, verbose=1)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
import pickle
pickle.dump(model, open('stack_model_submission-2.pkl','wb'))

In [ ]:
import pickle
model = pickle.load(open("stack_model_submission-2.pkl", "rb"))

In [ ]:
pred = model.predict(x_test)
print(f'test_data: {smape(y_test, pred)}')
plot_eval(pred, y_test)

In [ ]:
pred = model.predict(x_train)
print(f'train_data: {smape(y_train, pred)}')
plot_eval(pred, y_train)

In [ ]:
df_test = pd.read_csv('new_test.csv')
df_test.head()

In [ ]:
df_test = prepare_data(df_test)
df_test.head()

In [ ]:
cols = X.columns
data_test = df_test[cols]


pred_test = model.predict(data_test)

In [ ]:
subm = pd.read_csv('submissionV2.csv')
subm.head()

In [ ]:
subm['rerata_kecepatan'] = pred_test
subm.head()

In [ ]:
subm.to_csv('submission-new_data-v2.csv', index=False)